# **Web Scraping for ZipRecruiter**

## **Importing Libraries

In [1]:
#!pip install bs4
#!pip install numpy
#!pip install pandas

#!pip install ast
#!pip install matplotlib
#!pip install nltk
#!pip install wordcloud
!pip install xgboost

In [2]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import numpy as np
import nltk
import nltk
import os
import csv
nltk.download('punkt')
nltk.download('stopwords')
from nltk import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt
#from hcluster import pdist, linkage, dendrogram
from nltk.tokenize import RegexpTokenizer
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram
from sklearn.metrics import pairwise_distances

import scipy.cluster.hierarchy as sch



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Tyler\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Scraping Zip Recruiter for skills in data science job postings



Sraping Zip Recruiter for Job postings to find relevant skills required. Much difficulty scraping this website as it uses an "infinite scroll" system to load more results. Investigated the use of Selenium which can act as a human and click buttons on the page, but browser drivers were needed so considered this not an acceptable option for this project. In the end I decided to make a loop scrolling through 5 pages of each city (20 results per page) to gain the data as typically only 5 pages were available per city.

In [3]:
# #Scrapping for 100 data points (20 points per page = 5 pages)
# #create an empty DF and set pages = 6
# zip_table = pd.DataFrame()
# URL = 'https://www.ziprecruiter.com/Jobs/Data-Scientist/--in-Toronto,ON'
# #i = 1
# for i in range(1,20):
#     time.sleep(0.5)
# #conducting a request of the stated URL above:
#     if i == 1:
#         page = requests.get(URL)
#         #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
#         soup = BeautifulSoup(page.text, 'html.parser')
#     elif i > 1:
#         URL = 'https://www.ziprecruiter.com/Jobs/Data-Scientist/--in-Toronto,ON/'
#         URL+=str(i)
#         page = requests.get(URL)
#         #specifying a desired format of “page” using the html parser - this allows python to read the various components of the page, rather than treating it as one long string.
#         soup = BeautifulSoup(page.text, 'html.parser')

#     for jobs in soup.find_all("article", class_ = "job_result"): 
#       try:
#           job_title = jobs.find('span', class_ = 'just_job_title').text.strip()
#       #print(job_title)
#       except:
#           job_title = None

#       try:
#         location = jobs.find('span', class_ = 'location').text.strip()
#       except:
#         location = None

#       try:
#         company = jobs.find('span', class_ = 'name').text.strip()
#       except:
#         company = None

#       try:
#         link = jobs.find('p', attrs={'class': 'job_snippet'}).find('a', href=True)['href']
#       except:
#         link = None

#       try:
#         salary = jobs.find('span', attrs={'class': 'data_item'}).text.strip()
#       except:
#         salary = None    
    
#       zip_table = zip_table.append({
#           'job_title': job_title, 
#           'company': company,
#           'location': location,
#           'salary': salary,
#           'link': link}, ignore_index = True)
        


Below, a list of skills was made using terms found in the kaggle data science survey. Previously, links were obained by scraping Zip Recuiter as the job posting descriptions reside in a seprate URL. The new URLs are scraped below, and the regular expressions library is utilyzed to determine if the skills are mentioned in the job postings.

In [4]:

# skills = 'deep learning', 'machine learning', 'ai' ,'artificial intelligence', 'ml','data science', 'cost analysis','prediction','code','coding','data architecture','programming','machine learning','ai','statistics','mathematic','communication','critical thinking','visualization','master','professional','bachelor','business decisions','operationalizing','infrastructure','prototype','workflows','modeling','statistical','cloud','jupyter','rstudio','pycharm','atom','matlab','visual studio','spyder','vim','notepad','sublime','google colab','google','google cloud','notebook','ai platform','datalab','binder','ibm watson','ibm watson studio','ocean','python','r','sql','c','c++','java','javascript','typescript','bash','ggplot','matplotlib','plotly','d3','shiny','seaborn','geoplotlib','leaflet','cpu','gpu','tpu','linear','logistic','regression','forest','gradient','bayesian','neural networks','neural','dense','convolutional','transformer','ml','augmentation','automated','feature engineering','selection','architecture','','pipelines','image','video','pil','cv2','skimage','u-net','mask r-cnn','image segmentation','object detection  ','yolov3','retinanet','image classification','vgg','inception','resnet','gan','vae','computer vision','generative networks','word embeddings','vectors','glove','fasttext','word2ve','encoder','decorder seq2seq','vanilla','contextualized embeddings','elmo','cove','gpt-2','bert','xlnet','scikit-learn','tensorflow','keras','randomforest','xgboost','pyTorch','caret','lightgbm','spark mlib','fast.ai','amazon','azure','salesforce','alibaba','aws'
# #list of skills]

In [5]:
# zip_table['descript']='text'
# for skill in skills:
#     zip_table[skill] = np.zeros(len(zip_table))
# #reset_index
# zip_table = zip_table.reset_index()
# zip_table.head(5)

In [6]:
# tokenized=[]
# for i in range(len(zip_table)):
#     detail = requests.get(zip_table.iloc[i,3]) 
#     soup = BeautifulSoup(detail.text, "lxml")
    
#     #BELOW IS OPTION 2 (CAN NOT HANDLE MULTIPLE WORD SKILLS)
# #     try:
# #         text = soup.find("div", {'class' : 'job_details_container'}).text.strip().lower()
# #       # Remove the html tags
# #         re.sub(r'<[^<]+?>','', text)
# #   # Tokenize the text
# #     except:
# #         text = None
# #     zip_table.iloc[i,6] = text
# #     tokenizer = RegexpTokenizer('\W+',gaps=True)
# #     text=tokenizer.tokenize(str(text))
# #   # Extract the common skills
# #     common_value=list(set(skills).intersection(text))
# #     for word in common_value:
# #         zip_table.loc[i, word]=1
#   #get the text without none
#     try:
#         text = soup.find("div", {'class' : 'job_details_container'}).text.strip().lower()
#         string = re.sub(r'\,', ' ', text) # remove ","
#         #string = re.sub(r'\n',' ',string) 
#         string = re.sub(r'\,', ' ', string) # remove ","
#         string = re.sub('/', ' ', string) # remove "/"
#         string = re.sub(r'\(', ' ', string) # remove "("
#         string = re.sub(r'\)', ' ', string) # remove ")"
#         string = re.sub(' +',' ',string) # remove more than one space 
#         print(string)
        
#     except:
#         text = None
#         text = "" 
#     #zip_table.iloc[i,6] = string
#     for sk in skills :
#         if any(x in sk for x in ['+']):
#             skk = re.escape(sk)
#         else:
#             skk = sk
#         result = re.search(r'(?:^|(?<=\s))' + skk + r'(?=\s|$)',text)
        
#         if result:

#             zip_table[sk][i] = 1

In [7]:
# #zip_table

# zip_table = zip_table[zip_table.descript != "none"]
# zip_table = zip_table.drop_duplicates(keep='first')
# zip_table=zip_table.fillna(0)
# #zip_table = zip_table.loc[:, (zip_table != 0).any(axis=0)]
# #zip_table.to_csv(r'C:\Users\Tyler\Documents\Python notes\Project\ZipRecruiter.csv', index = False)
# zip_table.head()

Below, an attempt was made to obtain the most frequently used word across descriptions to mine for skills. Even with the StopWords libary, there were to many general terms used and no skills were obtained. It is clear that a large list of terms needs to be created in order to find the skills with highest importance (frequency). This was done in the above work.

In [8]:
#zip_table.to_csv(r'C:\Users\Tyler\Documents\Python notes\Project\ZipRecruiter.csv', index = False)
zip_table = pd.read_csv(r'./ZipRecruiter.csv')
zip_table.head()

# wf = zip_table.descript.str.cat(sep=' ')
# #function to split text into word
# tokens = word_tokenize(wf)
# vocabulary = set(tokens)
# print(len(vocabulary))
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# tokens = [w for w in tokens if not w in stop_words]
# frequency_dist = nltk.FreqDist(tokens)
# sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]

,index,company,job_title,link,location,salary,descript,deep learning,machine learning,ai,...,pyTorch,caret,lightgbm,spark mlib,fast.ai,amazon,azure,salesforce,alibaba,aws
0,0,rockwell automation,Software Data Architect,https://www.ziprecruiter.com/eclk/fAQRWYLiCODH...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,PagerDuty,Senior Data Engineer - Analytics,https://www.ziprecruiter.com/eclk/PrjVFPG3P6gb...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Goodfood Market Corp.,BI Engineer,https://www.ziprecruiter.com/c/Goodfood-Market...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Interac,Senior Data Engineer,https://www.ziprecruiter.com/eclk/gptSw66Ty4w_...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Project X,Consultant- Big Data Developer (4 Positions),https://www.ziprecruiter.com/c/Project-X/Job/C...,"Toronto, ON",Full-Time,text,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


It can be seen that the top 500 words are not relevant towards finding skills wanted in the field of data science.